<a href="https://colab.research.google.com/github/sreekarannaluru/Wildfire-Competition/blob/main/Merge_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working on merging datasets to put into the code


In [ ]:
#Denver area

df1 = pd.read_excel("/content/allison_cleaned_groupby_month_denver (1).xlsx")

In [ ]:
df2 = pd.read_excel("/content/modis_groupby_month_Denver (1).xlsx")

In [ ]:
df1.sample(3)

,Year,Month,Lat,Lon,IncidentSize,DiscoveryAcres,InitialResponseAcres,Occurences,SurfTemp,NDVI,SurfPS,Dust,SoilMoisture,WindVelocity,LAI,Canopy_h2o,Transpire
46,2017,11,-105.2100,39.536667,0.1,0.100,0.1000,3,274.3261,0.389793,80180.19,0.005364,23.86521,6.568827,1.054356,0.072161,0.472489
73,2020,2,-105.3075,39.570000,76.0,0.575,0.3625,6,263.1938,0.280041,80001.22,0.008610,39.15659,7.039024,0.570693,0.389887,0.000000
14,2015,3,-105.4950,39.640000,0.0,0.000,0.0000,2,274.8165,0.345370,80314.05,0.016662,29.92896,6.092209,0.614429,0.129921,0.204728


In [ ]:
denver_merged_df = df1.merge(df2, on = ["Year", "Month"], how = "left")

In [ ]:
denver_merged_df.to_excel("denver_without_weather_df.xlsx")

In [ ]:
df_d = pd.read_excel("/content/denver_without_weather_df.xlsx")

In [ ]:
#Columbs Area


In [ ]:
df1 = pd.read_excel("/content/allison_cleaned_groupby_month_columbus (1).xlsx")

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
df2 = pd.read_excel("/content/modis_groupby_month_Columbus (1).xlsx")

In [ ]:
columbus_merged_df = df1.merge(df2, on = ["Year", "Month"], how = "left" )

In [ ]:
columbus_merged_df.to_excel("columbus_without_weather_df.xlsx")

In [ ]:
df_c = pd.read_excel("/content/columbus_without_weather_df.xlsx")

#### Final Data Merge

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_excel("/content/Denver Weather.xlsx")

In [ ]:
df2 = pd.read_excel("/content/Columbus Weather.xlsx")

In [ ]:
df_d = df1.groupby(["Year", "Month"]).mean()

In [ ]:
df_d.to_excel("denver_weather.xlsx")

In [ ]:
df_c = df2.groupby(["Year", "Month"]).mean()

In [ ]:
df_c.to_excel("columbus_weather.xlsx")

In [ ]:
df_ww_d = pd.read_excel("/content/denver_without_weather_df.xlsx")

In [ ]:
df_ww_c = pd.read_excel("/content/columbus_without_weather_df.xlsx")

## Final data Set for the competition

In [ ]:
final_denver_df = df_ww_d.merge(df_d, on = ["Year", "Month"], how = "left")

In [ ]:
final_denver_df.to_excel("Denver.xlsx")

In [ ]:
final_columbus_df = df_ww_c.merge(df_c, on = ["Year", "Month"], how = "left")

In [ ]:
final_columbus_df.to_excel("Columbus.xlsx")

### Building NN model without weather data!

In [ ]:
# Load data for Denver
df_d = pd.read_excel('/content/denver_without_weather_df.xlsx')

In [ ]:
df_d["B"] = (df_d.brightness + df_d.bright_t31)/2

In [ ]:
df_d.columns

Index(['Year', 'Month', 'Lat', 'Lon', 'Occurences', 'SurfTemp', 'NDVI',
       'SurfPS', 'Dust', 'SoilMoisture', 'WindVelocity', 'LAI', 'Canopy_h2o',
       'Transpire', 'confidence', 'B'],
      dtype='object')

In [ ]:
#Swapping order
df_d = df_d[['Year', 'Month', 'Lat', 'Lon', 'Occurences', 'SurfTemp', 'NDVI',
       'SurfPS', 'Dust', 'SoilMoisture', 'WindVelocity', 'LAI', 'Canopy_h2o',
       'Transpire', 'B', 'confidence']]

In [ ]:
# Split into input and output variables (Regression)
X = data.iloc[:, 5:14].values # Input features: surface temperature, NDVI, wind velocity, etc.
y = data.iloc[:, 14].values # Output variable: brightness of the fire

In [ ]:
X.shape

(108, 9, 1, 1)

In [ ]:
# Reshape input data to 2D array for CNN
X = X.reshape(X.shape[0], 1, 1, X.shape[1])

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_size = (1,1,9), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10


ValueError: ignored

In [ ]:
# Predict the likelihood of a wildfire
new_data = np.array([[30, 60, 20, 5, 2, 0.5, 0.2, 0, 0.1, 0.8]])
new_data = new_data.reshape(new_data.shape[0], 1, 1, new_data.shape[1])
prediction = model.predict(new_data)

print(prediction)